NamedTuple

In [1]:
from collections import namedtuple

DataIngestionArtifact = namedtuple("DataIngestionArtifact",
                                   ["feature_store_file_path", "metadata_file_path", "download_dir"])

In [4]:
DI = DataIngestionArtifact("path_for_data_ingestion", "path_for_metadata_file", "path_for_download_dir")

In [5]:
DI[0]

'path_for_data_ingestion'

In [6]:
DI.metadata_file_path

'path_for_metadata_file'

In [7]:
DI._fields

('feature_store_file_path', 'metadata_file_path', 'download_dir')